# Module 8: Car Following

Here we will combine multiple aspects of what we've learned to have our Jetbot follow another Jetbot. The goal is to have our robot detect an AprilTag attached to the back of another vehicle, then use a PD Controller to follow the position of the robot.

Below is our normal starter code for initializing all variables and packages. An image should appear to ensure it is working properly. Make sure to have the various values for your camera coefficient matrix, the side length of the AprilTags, and determine the distance behind the Jetbot you want to stay behind the other vehicle (in meters).

In [ ]:
import threading
import time
import cv2
import numpy as np
from jetbot import Robot, Camera, bgr8_to_jpeg
from IPython.display import display
import ipywidgets as widgets
import dt_apriltags

# === JetBot Setup ===
robot = Robot()
camera = Camera.instance()

# === AprilTag Setup ===
detector = dt_apriltags.Detector()
cam_mtx = [94.861, 119.511, 119.506, 114.969]  # fx, fy, cx, cy
AT_size = 0.065  # meters
desired_distance = 0.2  # meters to maintain

# === PID Controller ===
class PIDController:
    def __init__(self, Kp=0.4, Kd=0.2):
        self.Kp = Kp
        self.Kd = Kd
        self.last_error = 0.0
        self.last_time = time.time()

    def compute(self, error):
        current_time = time.time()
        dt = current_time - self.last_time
        de = (error - self.last_error) / dt if dt > 0 else 0.0

        output = self.Kp * error + self.Kd * de
        self.last_error = error
        self.last_time = current_time
        return output

# === UI Widgets ===
image_widget = widgets.Image(format='jpeg', width=300, height=300)
trans_label = widgets.Label(value="Translation Z: ---")
control_label = widgets.Label(value="Control Output: ---")
kp_input = widgets.FloatText(value=0.4, description='Kp')
kd_input = widgets.FloatText(value=0.2, description='Kd')

go_button = widgets.Button(description="Go", button_style='success')
stop_button = widgets.Button(description="Stop", button_style='danger')
robot_enabled = False  # Start stopped

def on_go_clicked(b):
    global robot_enabled
    robot_enabled = True

def on_stop_clicked(b):
    global robot_enabled
    robot_enabled = False
    robot.stop()

go_button.on_click(on_go_clicked)
stop_button.on_click(on_stop_clicked)

display(widgets.VBox([
    image_widget,
    widgets.HBox([go_button, stop_button]),
    trans_label,
    control_label,
    kp_input,
    kd_input
]))

# === Control Loop ===
pid = PIDController(Kp=kp_input.value, Kd=kd_input.value)

def follow_apriltag():
    global robot_enabled
    while True:
        try:
            frame = np.array(camera.value)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            detections = detector.detect(gray, estimate_tag_pose=True, 
                                         camera_params=cam_mtx, tag_size=AT_size)

            if detections:
                detection = detections[0]  # Use first tag found
                pose_t = detection.pose_t  # (3, 1) translation vector

                # Distance in Z (forward direction)
                z_distance = pose_t[2][0]
                trans_label.value = f"Translation Z: {z_distance:.3f} m"

                # Calculate error to maintain 0.1 meters from tag
                error = z_distance - desired_distance

                # Update PID gains
                pid.Kp = kp_input.value
                pid.Kd = kd_input.value

                control = pid.compute(error)
                control_label.value = f"Control Output: {control:.3f}"

                if robot_enabled:
                    if z_distance > desired_distance:
                        # Move forward proportionally
                        speed = min(max(control, 0.0), 0.3)
                        robot.set_motors(speed, speed)
                    else:
                        robot.stop()
                else:
                    robot.stop()

                # Draw tag on image
                corners = detection.corners.astype(int)
                cv2.polylines(frame, [np.array(corners)], isClosed=True, color=(0, 255, 0), thickness=2)
                cv2.putText(frame, f"ID: {detection.tag_id}", (corners[0][0], corners[0][1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            else:
                trans_label.value = "Translation Z: ---"
                control_label.value = "Control Output: ---"
                robot.stop()

            image_widget.value = bgr8_to_jpeg(frame)

        except Exception as e:
            trans_label.value = f"Error: {str(e)}"
            control_label.value = "Control Output: ---"
            robot.stop()

        time.sleep(0.05)

# === Start Thread ===
threading.Thread(target=follow_apriltag, daemon=True).start()
